# I. Importation des bibliothèques

In [1]:
%%capture
!pip install --quiet fuzzywuzzy python-Levenshtein excel-formulas-calculator
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
%%capture
import warnings

warnings.filterwarnings("ignore")
import os
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
import math

import openpyxl as pyxl
from openhexa.sdk import workspace

try:
    from efc.interfaces.iopenpyxl import OpenpyxlInterface
except ImportError or ModuleNotFoundError:
    !pip install --quiet excel-formulas-calculator
    from efc.interfaces.iopenpyxl import OpenpyxlInterface

## 1. Rafraîchissement des bibliothèques

In [3]:
%%capture
# Ajout du chemin d'accès pour l'importation des bibliothèques
os.chdir(Path(workspace.files_path, "Fichier Suivi de Stock/code/pipelines"))

from importlib import reload

import compute_indicators
import export_file_to_google_drive as ggdrive
import generate_stock_tracking_file as gstf
import refresh_stock_tracking_file as rstf

# Importation des réquêtes sql
from compute_indicators.queries import QUERY_ETAT_STOCK_PROGRAMME, QUERY_ETAT_STOCK_PERIPH
from database_operations import process_statut_prod, stock_sync_manager

# Reload modules
reload(compute_indicators)
reload(stock_sync_manager)
reload(gstf)
reload(rstf)
reload(ggdrive)

# II. Définition des paramètres

## 1. Variables requises pour l'actualisation du fichier de suivi de stock

1. **Mois de création du rapport** : Fourni en tant qu'entrée du pipeline.

2. **Année de conception du rapport**: Fourni en tant qu'entrée du pipeline.

3. **Programme** : Le programme concerné pour lequel le fichier de suivi de stock.

4. **Fichier Suivi de Stock** : Le fichier suivi de stock finalisé après la réunion mensuelle.

In [4]:
month_report, year_report, programme, fp_suivi_stock = (
    "Mars",
    2025,
    "PNLP",
    "Fichier Suivi de Stock PNLP-MARS-2025.xlsx",
)

In [5]:
month_export, date_report = month_report, compute_indicators.utils.format_date(
    month_report, year_report
)

## 2. Test pour s'assurer qu'il y a bien des données déjà présentes dans la base de données pour ce programme en question

L'idée est de vérifier au préalable, avant d'apporter des modifications au programme en question, que des données existent déjà dans la base de données.

In [6]:
stock_sync_manager.initialize_database_connection()

schema_name = "suivi_stock"

Connexion à la base de données établie avec succès


In [7]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select * 
    from {schema_name}.stock_track st 
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    limit 2
    """
)

assert (
    df_.shape[0] != 0
), f"Le mois séléecitonné {date_report} n'a pas de données présente dans la base de données"

del df_

# III.📥Importation des Données
  
L'utilisateur doit veiller à ce que les fichiers respectent le format attendu et soient placés dans les répertoires dédiés avant de procéder au traitement.

## 📌1. Importation du fichier `Fichier Suivi des Stocks`

- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire dédié :  
  **`Fichier Suivi de Stock/data/<programme>/Fichier Suivi de Stock`**
  
- **En cas d'erreur :**  
  - Vérifiez que le fichier est bien présent dans le répertoire.  
  - Assurez-vous que toutes les colonnes requises sont bien renseignées.  
  - Contrôlez que le fichier est bien accessible et non corrompu.

In [8]:
fp_suivi_stock = Path(fp_suivi_stock)
# (
#    Path(workspace.files_path)
#    / f"Fichier Suivi de Stock/data/{programme}/Fichier Suivi de Stock"
#    / Path(fp_suivi_stock).name
# )
src_wb = pyxl.load_workbook(fp_suivi_stock)
sheetnames = src_wb.sheetnames

## 📌2. Importation de la feuille `Etat de stock de la NPSP`

In [9]:
df_etat_stock_npsp = rstf.get_data_from_sheet(
    fp_suivi_stock=fp_suivi_stock,
    sheet_name="Etat de stock",
    sheetnames=sheetnames,
    date_report=date_report,
    programme=programme,
    src_wb=src_wb,
)

In [11]:
df_etat_stock_npsp.head(2)

,date_report,code_produit,designation,contenance,dmm,traceurs,stock_theorique_bke,stock_theorique_abj,stock_theorique_central,stock_theorique_fin_mois,msd,statut_stock,programme
1,2025-03-01,3010049.0,PARACETAMOL 100 mg comp. BTE/100,BOITE/100,5357.166667,1.0,21379.0,71018.0,0.0,92397.0,17.247363,Surstock,PNLP
2,2025-03-01,3010062.0,PARACETAMOL 250 mg comp BTE/100,BOITE/100,6458.166667,1.0,13877.0,25574.0,0.0,39451.0,6.1087,Bon,PNLP


## 📌3. Importation de la feuille `Stock detaille`

In [12]:
df_stock_detaille = rstf.get_data_from_sheet(
    fp_suivi_stock=fp_suivi_stock,
    sheet_name="Stock detaille",
    sheetnames=sheetnames,
    date_report=date_report,
    programme=programme,
    src_wb=src_wb,
)

In [13]:
df_stock_detaille.head(2)

,Code produit,Désignation,Emplacement,Date limite de consommation,Numéro Lot,Sous lot (Programme),Qté \nPhysique,Qté \nlivrable,Unit,Jour restants,Code Couleur
0,4150558,MILDA STANDARD/DELTAMETHRINE,27Q00A00,2099-01-01,NEANT,PNLP,3245,0,UN,26882,GREEN
1,4150558,MILDA STANDARD/DELTAMETHRINE,27Q00A00,2099-01-01,NEANT,PNLP,139500,0,UN,26882,GREEN


## 📌4. Importation de la feuille `Distribution X3`

In [14]:
df_distribution = rstf.get_data_from_sheet(
    fp_suivi_stock=fp_suivi_stock,
    sheet_name="Distribution X3",
    sheetnames=sheetnames,
    date_report=date_report,
    programme=programme,
    src_wb=src_wb,
)

In [15]:
df_distribution.head(2)

,Programme,Date commande,No commande,Date livraison,No livraison,Référence,Client,Raison sociale,Article,Désignation,Unité vente,Qté commandée,Quantité livrée
0,PNLP,NaN,VAL01-PSG-2502-0230,2025-03-03,BL01-PSG-2503-0187,NaN,42400024,HOPITAL GENERAL GUEYO,3050016,ARTESUNATE/AMODIAQUINE 50 / 135 mg ENFANT (1 -...,BTE,1,1
1,PNLP,NaN,VAL01-PSG-2502-0230,2025-03-03,BL01-PSG-2503-0187,NaN,42400024,HOPITAL GENERAL GUEYO,3050062,ARTEMETHER/LUMEFANTRINE 20 / 120 mg (3 - 8 ANS...,BTE,21,21


## 📌5. Importation de la feuille `Receptions`

In [16]:
df_receptions = rstf.get_data_from_sheet(
    fp_suivi_stock=fp_suivi_stock,
    sheet_name="Receptions",
    sheetnames=sheetnames,
    date_report=date_report,
    programme=programme,
    src_wb=src_wb,
)

df_receptions["Date_entree_machine"] = pd.to_datetime(
    df_receptions["Date d'entrée en machine"], format="%d/%m/%Y", errors="coerce"
)

df_receptions["Nouveau code"] = pd.to_numeric(
    df_receptions["Nouveau code"], errors="coerce"
).astype(float)

In [17]:
df_receptions.head(2)

,Programme,Bailleur,Nouveau code,Code,Désignation NPSP,Date de réception effective,Date effective de dépotage et/ou expertise,Quantité réceptionnée,Date d'entrée en machine


## 📌6. Importation de la feuille `PPI`

In [18]:
df_ppi = rstf.get_data_from_sheet(
    fp_suivi_stock=fp_suivi_stock,
    sheet_name="PPI",
    sheetnames=sheetnames,
    date_report=date_report,
    programme=programme,
    src_wb=src_wb,
)

In [19]:
df_ppi.head(2)

,Code Produit,Nom Produit,Unite,Numéro Lot,Date Peremption,Quantité
0,3050015,AMODIAQUINE/ARTESUNATE 25 / 67.5 mg ENFANT (2 ...,BTE,EAG523001B,2025-01-31,25
1,3050016,AMODIAQUINE/ARTESUNATE 50 / 135 mg ENFANT (1 -...,BTE,EAC723001A,2025-01-31,18


## 📌7. Importation de la feuille  `Prélèvement`

In [20]:
df_prelevement = rstf.get_data_from_sheet(
    fp_suivi_stock=fp_suivi_stock,
    sheet_name="Prelèvement CQ",
    sheetnames=sheetnames,
    date_report=date_report,
    programme=programme,
    src_wb=src_wb,
)

In [21]:
df_prelevement.head(2)

,Code Produit,Nom Produit,Unite,Numéro Lot,Date Peremption,Quantité


## 📌8. Importation de la feuille `Plan d'approvisionnement`

In [22]:
df_plan_approv = rstf.get_data_from_sheet(
    fp_suivi_stock=fp_suivi_stock,
    sheet_name="Plan d'appro",
    sheetnames=sheetnames,
    date_report=date_report,
    programme=programme,
    src_wb=src_wb,
)

In [23]:
df_plan_approv.head(2)

,Standard product code,ID de produit QAT,Produits,ID de l`envoi QAT,Centrale d'achat,Source Financement,Status,Quantite,Facteur de conversion de QAT vers SAGE,Quantité harmonisée (SAGE),DATE,Cout des Produits,Couts du fret,Couts totaux,Acronym,Received?,Coût unitaire moyen (en dollar),Coût unitaire harmonisé,Date updated
0,4030209.0,3340,Giemsa Stain Solution 500 mL,160768,UNICEF,GFATM,Expédié,1071.0,1,1071.0,2025-05-31,11416.86,1369.93,12786.79,GIEMSA (500),0,10.660085,10.660085,mai-2025
1,4030224.0,3651,Immersion Oil 250 mL,160773,UNICEF,GFATM,Reçu,205.0,1,205.0,2025-04-10,3995.45,479.50,4474.95,HUILE A IMMERSION FL/250 ML FLACON,1,19.490000,19.490000,avril-2025


In [ ]:
# Extraction des version et des date maximale 
df_pa_version = stock_sync_manager.get_table_data(
    query=f"""select max(version_pa) version_pa, max(date_extraction_pa) date_extraction_pa
    from suivi_stock.plan_approv where date_report = '{date_report}' and programme='{programme}'; """
)

df_plan_approv = df_plan_approv.join(df_pa_version, how="cross")

del df_pa_version

## 📌9. Importation de la feuille `Statut Produits`

In [24]:
df_statut_prod = rstf.get_data_from_sheet(
    fp_suivi_stock=fp_suivi_stock,
    sheet_name="Statut Produits",
    sheetnames=sheetnames,
    date_report=date_report,
    programme=programme,
    src_wb=src_wb,
)

In [25]:
df_statut_prod.head(2)

,CODE,Ancien code,CATEGORIE,DESIGNATION DU PRODUIT,Type,Unité niv Central,Unité niv Périphérique,Facteur de conversion \n(De la centrale à la périphérie),Statut Produit,programme


## 📌10. Importation de la feuille `Annexe 1 - Consolidation`

In [26]:
df_etat_stock = rstf.get_data_from_sheet(
    fp_suivi_stock=fp_suivi_stock,
    sheet_name="Annexe 1 - Consolidation",
    sheetnames=sheetnames,
    date_report=date_report,
    programme=programme,
    src_wb=src_wb,
)

In [28]:
df_etat_stock.head(2)

,code_produit,Ancien code,CATEGORIE,DESIGNATION DU PRODUIT,Type,Unité niv Central,Unité niv Périphérique,Facteur de conversion \n(De la centrale à la périphérie),stock_theorique_mois_precedent,Distribution effectuée,Quantité reçue entrée en stock,Quantité de PPI,Quantité prélévée en Contrôle Qualité (CQ),Ajustement de stock,Stock Théorique Final SAGE,Stock Théorique Final Attendu,ECARTS,Justification des écarts,Diligences,Dilig. Choisie
0,3010049,AY13071,Médicaments,PARACETAMOL 100 mg comp. bte / 100,Non traceur,BOITE/100,COMPRIME,100,95338,2835,0,0,0,NaN,92397,92503,-106,NaN,NaN,NaN
1,3010062,AY13115,Médicaments,PARACETAMOL 250 mg comp bte / 100,Non traceur,BOITE/100,COMPRIME,100,43217,3484,0,0,0,NaN,39451,39733,-282,NaN,NaN,NaN


# IV. Mise à jour des informations

## 1. Mise à jour des informations sur le produit en se basant sur la feuille `Statut Produits`

À l’issue des réunions mensuelles, des ajustements peuvent être effectués sur les informations de produits.
Ces ajustements incluent :

 * l’ajout de nouveaux produits,
 * la suppression de produits existants,
 * la modification des attributs ou métadonnées associées à certains produits.

La fonction suivante a pour objectif de journaliser ces opérations afin d'assurer une traçabilité complète des modifications apportées.

In [18]:
process_statut_prod(df_statut_prod, schema_name, stock_sync_manager)

## 2. Mise à jour des informations Produits en se basant sur la feuille `Plan d'appro`

L'idée est de pouvoir faire des ajustements directement des informations sur les produits en se basant sur la feuille `Plan d'appro` les informations concernées sont:

* La désignation acronym des produits;
* Le facteur de conversion de QAT vers SAGE;
* Le coût unitaire moyen en dollar qui résulte d'un calcul analytique sur une périodicité des extractions QAT.

In [19]:
df_pa = df_plan_approv.rename(
    columns={
        "Acronym": "acronym",
        "Facteur de conversion de QAT vers SAGE": "facteur_de_conversion_qat_sage",
    }
)

In [20]:
stock_sync_manager.synchronize_product_metadata(df_pa, programme)

In [21]:
del df_pa

## 3. Recherche des modifications sur les `DMM`

In [29]:
df_stock_track_dmm, df_stock_track_dmm_histo = rstf.get_dmm_dataframes(
    df_etat_stock=df_etat_stock, src_wb=src_wb, sheetnames=sheetnames, date_report=date_report
)

## 4. Recherche des modifications sur les `CMM`

In [32]:
eomonth = (pd.to_datetime(date_report) + pd.offsets.MonthEnd(0)).strftime("%Y-%m-%d")

df_stock_prog_nat = stock_sync_manager.get_table_data(
    query=QUERY_ETAT_STOCK_PROGRAMME.format(eomonth=eomonth, programme=programme)
)

df_stock_prog_nat["Code_produit"] = df_stock_prog_nat["Code_produit"].astype(int)

df_stock_prog_nat.head(3)

,Programme,MSD,STATUT,CONSO,SDU,CMM,dispo_globale,dispo_globale_cible,dispo_traceur,dispo_traceur_cible,date_report,id_produit_fk,Code_region,region_order,statut_pourcentage,id_produit_pk,Code_produit,Produit_designation,Unit_rapportage,Categorie_produit,Categorie_du_produit,Code_sous_prog
0,PNLP,"1,0",SOUS-STOCK,31789,62881,63895.0,0.018922,0.023611,0.026342,0.026389,2025-03-31,17,NAT,None,0.027778,17,3050061,ARTEMETHER/LUMEFANTRINE 20 / 120 mg (0 - 3ANS)...,PLAQUETTE,Produit non traceur,PRODUITS PNLP,PNLP-1
1,PNLP,"0,7",SOUS-STOCK,2475,12450,16819.5,0.018922,0.023611,0.026342,0.026389,2025-03-31,4882,NAT,None,0.027778,4882,3010103,AMODIAQUINE 75 MG + SULFADOXINE/PYRIMETHAMINE ...,COMPRIME,Produit non traceur,MEDICAMENTS,PNLP-1
2,PNLP,"4,7",SURSTOCK,28635,154550,32958.5,0.018922,0.023611,0.026342,0.026389,2025-03-31,5310,NAT,None,0.027778,5310,4151200,MILDA Dual AI BALLE/50 BALLE -,MOUSTIQUAIRE,Produit non traceur,MEDICAMENTS,PNLP-1


In [33]:
df_stock_track_cmm, df_stock_track_cmm_histo = rstf.get_cmm_dataframes(
    df_etat_stock=df_etat_stock,
    df_stock_prog_nat=df_stock_prog_nat,
    src_wb=src_wb,
    sheetnames=sheetnames,
    date_report=date_report,
)

## 5. Recherche des modifications sur `Annexe 2 - Suivi des Stocks`

In [37]:
df_etat_stock = rstf.get_data_etat_stock(
    src_wb=src_wb,
    sheetnames=sheetnames,
    df_etat_stock=df_etat_stock,
    df_stock_prog_nat=df_stock_prog_nat,
    df_plan_approv=df_plan_approv,
    date_report=date_report,
)

# V.Exportation des données

In [40]:
dim_produit = pd.read_sql(
    f"SELECT * FROM {schema_name}.dim_produit_stock_track where programme='{programme}'",
    stock_sync_manager.civ_engine,
)

## 1. `Etat de Stock`

In [41]:
df_etat_stock = df_etat_stock.merge(
    dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]],
    on="code_produit",
    how="inner",
).rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})[
    [
        "id_dim_produit_stock_track_fk",
        "stock_theorique_mois_precedent",
        "distribution_effectuee",
        "quantite_recue_stock",
        "quantite_ppi",
        "quantite_prelevee_cq",
        "ajustement_stock",
        "stock_theorique_final_sage",
        "stock_theorique_final_attendu",
        "ecarts",
        "justification_ecarts",
        "diligences",
        "sdu_central_annexe_2",
        "dmm_central_annexe_2",
        "msd_central_annexe_2",
        "statut_central_annexe_2",
        "conso_decentralise_annexe_2",
        "sdu_decentralise_annexe_2",
        "cmm_decentralise_annexe_2",
        "msd_decentralise_annexe_2",
        "statut_decentralise_annexe_2",
        "nombre_de_site_en_rupture_annexe_2",
        "sdu_national_annexe_2",
        "cmm_national_annexe_2",
        "msd_national_annexe_2",
        "statut_national_annexe_2",
        "date_peremption_plus_proche_brute_annexe_2",
        "date_peremption_plus_proche_annexe_2",
        "quantite_correspondante_annexe_2",
        "msd_correspondant_annexe_2",
        "quantite_attendue_annexe_2",
        "msd_attendu_annexe_2",
        "quantite_non_stockee_annexe_2",
        "msd_recu_annexe_2",
        "financement_annexe_2",
        "date_probable_livraison_annexe_2",
        "date_effective_livraison_annexe_2",
        "statut_annexe_2",
        "analyse_risque_commentaires_annexe_2",
        "diligences_central_annexe_2",
        "diligences_peripherique_annexe_2",
        "responsable_annexe_2",
        "dilig_choisie_annexe_2",
        "date_report",
    ]
]

for col in [col for col in df_etat_stock.columns if "msd" in col]:
    df_etat_stock[col] = df_etat_stock[col].apply(
        lambda x: str(round(float(x), 1)).replace(".", ",")
        if not pd.isna(x) and x != "ND" and x != "NA" and x != ""
        else x
    )

In [42]:
df_etat_stock["date_peremption_plus_proche_annexe_2"] = pd.to_numeric(
    df_etat_stock["date_peremption_plus_proche_annexe_2"], errors="coerce", downcast="integer"
)

df_etat_stock["date_peremption_plus_proche_annexe_2"] = pd.to_datetime(
    df_etat_stock["date_peremption_plus_proche_annexe_2"],
    unit="D",
    origin="1899-12-30",
    errors="coerce",
)

In [53]:
for col in [col for col in df_etat_stock.columns if "date_" in col]:
    # df_etat_stock[col] = df_etat_stock[col].astype("datetime64[ns]")
    df_etat_stock[col] = df_etat_stock[col].apply(
        lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d")
        if len(str(x)) >= 10
        else np.nan
    )

In [54]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    """
)

for col in [col for col in df_.columns if "date_" in col]:
    df_[col] = df_[col].astype("datetime64[ns]")

In [55]:
for col in df_etat_stock.columns:
    if col in df_.columns and df_etat_stock[col].dtype != df_[col].dtype:
        try:
            df_etat_stock[col] = df_etat_stock[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_etat_stock[col] = pd.to_numeric(df_etat_stock[col], downcast=df_[col].dtype, errors="coerce")

In [56]:
df_etat_stock = df_etat_stock.replace({pd.NaT: None})

In [81]:
stock_sync_manager.upsert_dataframe(
    df=df_etat_stock,
    table_name="stock_track",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=["id_dim_produit_stock_track_fk", "date_report"],
)

'Upsert de 39 enrégistrements réussie'

## 2. `Distributions`

### 2.1.`Distributions mois courant` 

In [58]:
df_stock_track_dmm = (
    df_stock_track_dmm.merge(
        dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]],
        on="code_produit",
        how="inner",
    )
    .drop(columns="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
)

df_stock_track_dmm.head(3)

,date_report,dmm,nbre_mois_consideres,distributions_mois_consideres,dmm_calculee,commentaire,id_dim_produit_stock_track_fk
0,2025-03-01,2835,6.0,30020.0,5003.333333,None,14
1,2025-03-01,3484,6.0,35547.0,5924.500000,None,15
2,2025-03-01,255,6.0,12014.0,2002.333333,None,1


In [60]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track_dmm st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    limit 10
    """
)

In [61]:
for col in df_stock_track_dmm.columns:
    if col in df_.columns and df_stock_track_dmm[col].dtype != df_[col].dtype:
        try:
            df_stock_track_dmm[col] = df_stock_track_dmm[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_stock_track_dmm[col] = pd.to_numeric(
                    df_stock_track_dmm[col], downcast=df_[col].dtype, errors="coerce"
                )

In [ ]:
id_list = list(map(int, df_stock_track_dmm.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_dmm
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [62]:
stock_sync_manager.upsert_dataframe(
    df=df_stock_track_dmm,
    table_name="stock_track_dmm",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=["id_dim_produit_stock_track_fk", "date_report"],
)

'Upsert de 39 enrégistrements réussie'

### 2.2.`Distributions hitoriques considérées pour le mois en cours`

In [63]:
df_stock_track_dmm_histo = (
    df_stock_track_dmm_histo.merge(
        dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]],
        on="code_produit",
        how="inner",
    )
    .drop(columns="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
)

df_stock_track_dmm_histo.head(3)

,date_report_prev,dmm,date_report,id_dim_produit_stock_track_fk
0,2024-09-01,4633,2025-03-01,14
1,2024-10-01,4181,2025-03-01,14
2,2024-11-01,8472,2025-03-01,14


In [64]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track_dmm_histo st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    limit 10
    """
)

In [65]:
for col in df_stock_track_dmm_histo.columns:
    if col in df_.columns and df_stock_track_dmm_histo[col].dtype != df_[col].dtype:
        try:
            df_stock_track_dmm_histo[col] = df_stock_track_dmm_histo[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_stock_track_dmm_histo[col] = pd.to_numeric(
                    df_stock_track_dmm_histo[col], downcast=df_[col].dtype, errors="coerce"
                )

In [ ]:
id_list = list(map(int, df_stock_track_dmm_histo.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_dmm_histo
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [66]:
stock_sync_manager.upsert_dataframe(
    df=df_stock_track_dmm_histo,
    table_name="stock_track_dmm_histo",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=[
        "id_dim_produit_stock_track_fk",
        "date_report",
        "date_report_prev",
    ],
)

'Upsert de 234 enrégistrements réussie'

## 3.`Consommations`

### 3.1. `Consommations mois courant`

In [67]:
df_stock_track_cmm = (
    df_stock_track_cmm.merge(
        dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]],
        on="code_produit",
        how="inner",
    )
    .drop(columns="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
)

df_stock_track_cmm.head(3)

,date_report,cmm,nbre_mois_consideres,conso_mois_consideres,cmm_calculee,commentaire,id_dim_produit_stock_track_fk
0,2025-03-01,5394,6.0,27932.0,4655.333333,None,14
1,2025-03-01,6782,6.0,33836.0,5639.333333,None,15
2,2025-03-01,1760,6.0,11978.0,1996.333333,None,1


In [68]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track_cmm st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    limit 2
    """
)

In [69]:
for col in df_stock_track_cmm.columns:
    if col in df_.columns and df_stock_track_cmm[col].dtype != df_[col].dtype:
        try:
            df_stock_track_cmm[col] = df_stock_track_cmm[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_stock_track_cmm[col] = pd.to_numeric(
                    df_stock_track_cmm[col], downcast=df_[col].dtype, errors="coerce"
                )

In [ ]:
id_list = list(map(int, df_stock_track_cmm.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_cmm
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [70]:
stock_sync_manager.upsert_dataframe(
    df=df_stock_track_cmm,
    table_name="stock_track_cmm",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=["id_dim_produit_stock_track_fk", "date_report"],
)

'Upsert de 39 enrégistrements réussie'

### 3.2.`Consommations hitoriques considérées pour le mois en cours`

In [71]:
df_stock_track_cmm_histo = (
    df_stock_track_cmm_histo.merge(
        dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]],
        on="code_produit",
        how="inner",
    )
    .drop(columns="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
)

df_stock_track_cmm_histo.head(3)

,date_report_prev,cmm,date_report,id_dim_produit_stock_track_fk
0,2024-10-01,4616,2025-03-01,14
1,2024-11-01,4812,2025-03-01,14
2,2024-12-01,4700,2025-03-01,14


In [72]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track_cmm_histo st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    limit 2
    """
)

In [129]:
for col in df_stock_track_cmm_histo.columns:
    if col in df_.columns and df_stock_track_cmm_histo[col].dtype != df_[col].dtype:
        try:
            df_stock_track_cmm_histo[col] = df_stock_track_cmm_histo[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_stock_track_cmm_histo[col] = pd.to_numeric(
                    df_stock_track_cmm_histo[col], downcast=df_[col].dtype, errors="coerce"
                )

In [ ]:
id_list = list(map(int, df_stock_track_cmm_histo.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_cmm_histo
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [130]:
stock_sync_manager.upsert_dataframe(
    df=df_stock_track_cmm_histo,
    table_name="stock_track_cmm_histo",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=[
        "id_dim_produit_stock_track_fk",
        "date_report",
        "date_report_prev",
    ],
)

'Upsert de 234 enrégistrements réussie'

## 4.`Stock détaillé`

In [132]:
code_col = [col for col in df_stock_detaille.columns if "CODE" in str(col).upper()][0]

df_stock_detaille.rename(
    columns={
        code_col: "code_produit",
        "Désignation": "designation_produit",
        "Emplacement": "emplacement",
        "Date limite de consommation": "date_limite_consommation",
        "Numéro Lot": "numero_lot",
        "Sous lot (Programme)": "sous_lot_programme",
        "Qté \nPhysique": "qte_physique",
        "Qté \nlivrable": "qte_livrable",
        "Unit": "unit",
    },
    inplace=True,
)
df_stock_detaille = df_stock_detaille[
    [
        "code_produit",
        "designation_produit",
        "emplacement",
        "date_limite_consommation",
        "numero_lot",
        "sous_lot_programme",
        "qte_physique",
        "qte_livrable",
        "unit",
    ]
]

df_stock_detaille = df_stock_detaille.loc[
    df_stock_detaille.code_produit.isin(dim_produit.code_produit)
]
df_stock_detaille.head(1)

,code_produit,designation_produit,emplacement,date_limite_consommation,numero_lot,sous_lot_programme,qte_physique,qte_livrable,unit
2,4030247,LAME PORTE OBJET BTE/50,01A01A01,2025-09-01,09323,PNLP,100,0,BTE


In [133]:
df_stock_detaille = (
    df_stock_detaille.groupby(["code_produit", "date_limite_consommation"])[
        ["qte_physique", "qte_livrable"]
    ]
    .sum(min_count=1)
    .reset_index()
)

df_stock_detaille = (
    dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]]
    .merge(df_stock_detaille, on="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
    .drop(columns=["code_produit"])
)

df_stock_detaille["date_report"] = pd.to_datetime(date_report)

In [135]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track_detaille st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    """
)

In [136]:
for col in df_stock_detaille.columns:
    if col in df_.columns and df_stock_detaille[col].dtype != df_[col].dtype:
        try:
            df_stock_detaille[col] = df_stock_detaille[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_stock_detaille[col] = pd.to_numeric(
                    df_stock_detaille[col], downcast=df_[col].dtype, errors="coerce"
                )

In [138]:
stock_sync_manager.upsert_dataframe(
    df=df_stock_detaille,
    table_name="stock_track_detaille",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=[
        "id_dim_produit_stock_track_fk",
        "date_limite_consommation",
        "date_report",
    ],
)

'Upsert de 67 enrégistrements réussie'

## 5. `Etat de stock de la NPSP`

Pour cette table ci il faut juste l'exporter vers la base de données

In [ ]:
query = f"""
DELETE FROM {schema_name}.stock_track_npsp
WHERE date_report = '{date_report}' and programme='{programme}';
"""

stock_sync_manager.civ_cursor.execute(query)
stock_sync_manager.conn.commit()

In [ ]:
stock_sync_manager.insert_dataframe_to_table(
    df_etat_stock_npsp, table_name="stock_track_npsp", schema_name="suivi_stock"
)

## 6.`Prévision`

In [144]:
import locale
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')
df_plan_approv['Date updated'] = df_plan_approv['DATE'].apply(lambda x: x.strftime("%b-%Y"))

In [146]:
df_prevision = compute_indicators.prevision.get_prevision_current_month(
    df_plan_approv.copy(),
    date_report,
    programme,
    stock_sync_manager.civ_engine,
    schema_name="suivi_stock",
)

,id_dim_produit_stock_track_fk,code_produit,ancien_code,stock_central,dmm_central,stock_national,cmm_national,stock_prev_central,stock_prev_national,period_prev,date_report
0,14,3010049,AY13071,92397.0,5003.3335,105553.0,4922.0,18,21,2025-03-01,2025-03-01
1,14,3010049,AY13071,NaN,NaN,NaN,NaN,17,20,2025-04-01,2025-03-01
2,14,3010049,AY13071,NaN,NaN,NaN,NaN,16,19,2025-05-01,2025-03-01


In [ ]:
id_list = list(map(int, df_prevision.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_prevision
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""
stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [ ]:
stock_sync_manager.insert_dataframe_to_table(
    df_prevision.drop(columns=["code_produit", "ancien_code"]), "stock_track_prevision"
)

## 7.`Plan d'approvisionnement`

In [ ]:
query = f"""
DELETE FROM {schema_name}.plan_approv
WHERE date_report = '{date_report}' and programme='{programme}'
"""
stock_sync_manager.civ_cursor.execute(query)
stock_sync_manager.conn.commit()

In [39]:
df_pa = df_plan_approv.rename(
    columns={
        "Standard product code": "standard_product_code",
        "ID de produit QAT": "id_produit_qat",
        "Produits": "designation",
        "ID de l`envoi QAT": "id_envoi_qat",
        "Centrale d'achat": "centrale_achat",
        "Source Financement": "source_financement",
        "Status": "status",
        "Quantite": "quantite",
        "Facteur de conversion de QAT vers SAGE": "facteur_conversion_qat_vers_sage",
        "Quantité harmonisée (SAGE)": "quantite_harmonisee_sage",
        "DATE": "date",
        "Cout des Produits": "cout_produits",
        "Couts du fret": "cout_fret",
        "Couts totaux": "cout_total",
    },
)

df_pa = df_pa[
    ["standard_product_code", "id_produit_qat",
     "designation", "id_envoi_qat",
     "centrale_achat", "source_financement",
     "status", "quantite", "facteur_conversion_qat_vers_sage",
     "quantite_harmonisee_sage", "date",
     "cout_produits", "cout_fret", "cout_total",
     "version_pa", "date_extraction_pa",
    ]
]
df_pa["date_report"] = pd.to_datetime(date_report)
df_pa["programme"] = programme
df_pa["standard_product_code"] = df_pa["standard_product_code"].fillna(0)

df_pa.head(2)

,standard_product_code,id_produit_qat,designation,id_envoi_qat,centrale_achat,source_financement,status,quantite,facteur_conversion_qat_vers_sage,quantite_harmonisee_sage,date,cout_produits,cout_fret,cout_total,date_report,programme
0,4030209.0,3340,Giemsa Stain Solution 500 mL,160768,UNICEF,GFATM,Expédié,1071.0,1,1071.0,2025-05-31,11416.86,1369.93,12786.79,2025-03-01,PNLP
1,4030224.0,3651,Immersion Oil 250 mL,160773,UNICEF,GFATM,Reçu,205.0,1,205.0,2025-04-10,3995.45,479.50,4474.95,2025-03-01,PNLP


In [ ]:
stock_sync_manager.insert_dataframe_to_table(df_pa, table_name="plan_approv", schema_name="suivi_stock")

del df_pa

In [ ]:
# Mise à jour des colonnes qui contienent des valeurs nan, NaN ou None malgré les mises à jours effectuée
from database_operations import queries

stock_sync_manager.civ_cursor.execute(
    queries.QUERY_UPDATE.format(schema_name=schema_name)
)
stock_sync_manager.conn.commit()

# VII. Exportation du Fichier Suivi de Stock vers le `drive` et la `BD`

In [ ]:
download_url = ggdrive.upload_and_return_link(
    fp_suivi_stock.as_posix(),
    date_report,
)

In [ ]:
df_download_url = pd.DataFrame(
    data=[
        {
            "programme": programme,
            "download_url": download_url,
            "date_report": date_report,
        }
    ]
)

df_download_url.date_report = df_download_url.date_report.apply(
    lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d")
)

In [ ]:
stock_sync_manager.synchronize_table_data(
    df_download_url,
    table_name="share_link",
    merge_keys=["programme", "date_report"],
    programme=programme,
)